In [ ]:
# Loading libraries
import pandas as pd
import numpy as np

In [ ]:
# Global constants and variables
TRAIN_FILENAME = 'train.csv'
TEST_FILENAME = 'test.csv'

In [ ]:
train = pd.read_csv('../input/'+TRAIN_FILENAME, parse_dates=['Dates'], index_col=False)
test = pd.read_csv('../input/'+TEST_FILENAME, parse_dates=['Dates'], index_col=False)

In [ ]:
train.info()

In [ ]:
train = train.drop(['Descript', 'Resolution', 'Address'], axis = 1)

In [ ]:
test = test.drop(['Address'], axis = 1)

In [ ]:
def feature_engineering(data):
    data['Day'] = data['Dates'].dt.day
    data['Month'] = data['Dates'].dt.month
    data['Year'] = data['Dates'].dt.year
    data['Hour'] = data['Dates'].dt.hour
    data['Minute'] = data['Dates'].dt.minute
    data['DayOfWeek'] = data['Dates'].dt.dayofweek
    data['WeekOfYear'] = data['Dates'].dt.weekofyear
    return data

In [ ]:
train = feature_engineering(train)

In [ ]:
test = feature_engineering(test)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
enc = LabelEncoder()
train['PdDistrict'] = enc.fit_transform(train['PdDistrict'])

In [ ]:
category_encoder = LabelEncoder()
category_encoder.fit(train['Category'])
train['CategoryEncoded'] = category_encoder.transform(train['Category'])
print(category_encoder.classes_)

In [ ]:
enc = LabelEncoder()
test['PdDistrict'] = enc.fit_transform(test['PdDistrict'])

In [ ]:
print(train.columns)
print(test.columns)

In [ ]:
x_cols = list(train.columns[2:12].values)
x_cols.remove('Minute')
print(x_cols)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 10)

In [ ]:
clf.fit(train[x_cols], train['CategoryEncoded'])

In [ ]:
test['predictions'] = clf.predict(test[x_cols])

In [ ]:
def field_to_columns(data, field, new_columns):
    for i in range(len(new_columns)):
        data[new_columns[i]] = (data[field] == new_columns[i]).astype(int)
    return data

In [ ]:
test['Category'] = category_encoder.inverse_transform(test['predictions'])

In [ ]:
categories = list(category_encoder.classes_)

In [ ]:
test = field_to_columns(test, 'Category', categories)

In [ ]:
import time
PREDICTIONS_FILENAME_PREFIX = 'predictions_'
PREDICTIONS_FILENAME = PREDICTIONS_FILENAME_PREFIX + time.strftime('%Y%m%d-%H%M%S') + '.csv'

In [ ]:
print(test.columns)

In [ ]:
submission_cols = [test.columns[0]]+list(test.columns[14:])
print(submission_cols)

In [ ]:
print(PREDICTIONS_FILENAME)
test[submission_cols].to_csv(PREDICTIONS_FILENAME, index = False)